In [121]:
# Cấu hình chung
#file wight của yolo
checkpoint:str ="./runs/train-silu/exp1/weights/best_ckpt" 
#Loại thiết bị
device:str = "cpu"#@param ["gpu", "cpu"]
half:bool = False #@param {type:"boolean"}
max_cosine_distance = 0.4
nn_budget = None
#Weigh của deepsort
model_filename = 'model_data/mars-small128.pb'

In [122]:
#Yolo import
import os, requests, torch, math, cv2
import numpy as np
import PIL
from yolov6.utils.events import LOGGER, load_yaml
from yolov6.layers.common import DetectBackend
from yolov6.data.data_augment import letterbox
from yolov6.utils.nms import non_max_suppression
from yolov6.core.inferer import Inferer
from typing import List, Optional

In [123]:
#Deep sort import
from deep_sort.tools import generate_detections as gdet
from deep_sort.application_util import preprocessing
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort.tracker import Tracker

In [124]:

#Tải xuống weight mặc định nếu config không tồn tại
if not os.path.exists(f"{checkpoint}.pt"):
  print("Downloading checkpoint...")
  os.system(f"""wget -c https://github.com/meituan/YOLOv6/releases/download/0.2.0/{checkpoint}.pt""")


In [125]:
#Khởi tạo deep_sort
encoder = gdet.create_box_encoder(model_filename, batch_size=1)
encoder = gdet.create_box_encoder(model_filename, batch_size=1)
metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
tracker = Tracker(metric)


In [126]:
#Set-up hardware options
cuda = device != 'cpu' and torch.cuda.is_available()
device = torch.device('cuda:0' if cuda else 'cpu')

In [127]:
def check_img_size(img_size, s=32, floor=0):
  def make_divisible( x, divisor):
    # Upward revision the value x to make it evenly divisible by the divisor.
    return math.ceil(x / divisor) * divisor
  """Make sure image size is a multiple of stride s in each dimension, and return a new shape list of image."""
  if isinstance(img_size, int):  # integer i.e. img_size=640
      new_size = max(make_divisible(img_size, int(s)), floor)
  elif isinstance(img_size, list):  # list i.e. img_size=[640, 480]
      new_size = [max(make_divisible(x, int(s)), floor) for x in img_size]
  else:
      raise Exception(f"Unsupported type of img_size: {type(img_size)}")

  if new_size != img_size:
      print(f'WARNING: --img-size {img_size} must be multiple of max stride {s}, updating to {new_size}')
  return new_size if isinstance(img_size,list) else [new_size]*2

def precess_image(rgb_img, img_size, stride, half):
  '''Process image before image inference.'''
  try:
    assert rgb_img is not None, f'Invalid image'
  except Exception as e:
    LOGGER.Warning(e)
  image = letterbox(rgb_img, img_size, stride=stride)[0]

  # Convert
  image = image.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
  image = torch.from_numpy(np.ascontiguousarray(image))
  image = image.half() if half else image.float()  # uint8 to fp16/32
  image /= 255  # 0 - 255 to 0.0 - 1.0

  return image, rgb_img



In [128]:
model = DetectBackend(f"./{checkpoint}.pt", device=device)
stride = model.stride
#class_names = load_yaml("./data/coco.yaml")['names']
class_names=["S", "I"]
if half & (device.type != 'cpu'):
  model.model.half()
else:
  model.model.float()
  half = False

# if device.type != 'cpu':
#   model(torch.zeros(1, 3, *img_size).to(device).type_as(next(model.model.parameters())))  # warmup

Loading checkpoint from ././runs/train-silu/exp1/weights/best_ckpt.pt

Fusing model...


In [129]:
class Node:
    def __init__(self,_x,_y,_id) -> None:
        self.x=_x,
        self.y=_y,
        self.len = 0
        self.id=_id
    def update(self,_x,_y):
        self.len = abs(_x-self.x) + abs(_y-self.y)
        self.x=_x,
        self.y=_y


In [130]:

import json
memo={}
def node_to_json(obj):
    if isinstance(obj, Node):
        return json.dumps({obj.id,obj.len})
def node_to_dict(obj):
    if isinstance(obj, Node):
        return float(obj.len)
def detect_per_frame(rgb_img, img_size=640,conf_thres=.24,iou_thres=.45,max_det=1000,agnostic_nms=False):
  img_size = check_img_size(img_size, s=stride)
  img, img_src = precess_image(rgb_img, img_size, stride, half)
  img = img.to(device)
  if len(img.shape) == 3:
      img = img[None]
      # expand for batch dim
  pred_results = model(img)
  classes:Optional[List[int]] = None # the classes to keep
  det = non_max_suppression(pred_results, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)[0]
  gn = torch.tensor(img_src.shape)[[1, 0, 1, 0]]  # normalization gain whwh
  img_ori = img_src.copy()
  
  if len(det):
    det[:, :4] = Inferer.rescale(img.shape[2:], det[:, :4], img_src.shape).round()
    bboxes = []
    scores = []
    for *xyxy, conf, cls in reversed(det):
        bbox_left = min([xyxy[0].item(), xyxy[2].item()])
        bbox_top = min([xyxy[1].item(), xyxy[3].item()])
        bbox_w = abs(xyxy[0].item() - xyxy[2].item())
        bbox_h = abs(xyxy[1].item() - xyxy[3].item())
        box = [bbox_left, bbox_top, bbox_w, bbox_h]
        if bbox_w <=0 or bbox_h<=0:
           print(xyxy)
           continue
        bboxes.append(box)
        scores.append(conf.item())
        
       
    if len(bboxes)<1:
       print('No object')
       return img_ori
    # DeepSORT -> Getting appearence features of the object.
    features = encoder(img_src, bboxes)
    # DeepSORT -> Storing all the required info in a list.
    detections = [Detection(bbox, score, feature) for bbox, score, feature in zip(bboxes, scores, features)]

    # DeepSORT -> Predicting Tracks. 
    tracker.predict()
    tracker.update(detections)
    # DeepSORT -> Plotting the tracks.
    # print( "Total = "+ str(len(tracker.tracks)))
    # print("is_confirmed = " + str( sum([int(i.is_confirmed()) for i in tracker.tracks])))
    # print("is_tentative = " + str(sum([int(i.is_tentative()) for i in tracker.tracks])))
    # print("is_deleted = " + str(sum([int(i.is_deleted()) for i in tracker.tracks])))

    for track in tracker.tracks:
      if not track.is_confirmed():
          continue
      # DeepSORT -> Changing track bbox to top left, bottom right coordinates
      bbox = list(track.to_tlbr())
      # DeepSORT -> Writing Track bounding box and ID on the frame using OpenCV.
      txt = 'id:' + str(track.track_id)
      if memo.get(track.track_id) is not None:
         memo.get(track.track_id).update(bbox[0],bbox[1])
      else:
         memo[track.track_id]= Node(bbox[0] ,bbox[1],track.track_id)
      #cv2.putText(img_ori,txt,(int(bbox[0]),int(bbox[1])),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2,cv2.LINE_AA)
      Inferer.plot_box_and_label(img_ori, max(round(sum(img_ori.shape) / 2 * 0.003), 2), bbox, txt, color=Inferer.generate_colors(1, True))
    return img_ori,memo

In [131]:
import cv2
import numpy as np
def detect_from_video(path,output_video='output1.mp4'):
    cap = cv2.VideoCapture(path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  
    # Tạo đối tượng VideoWriter để tạo video đầu ra
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc,25, (frame_width, frame_height))

    # Check if camera opened successfully
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")
    frame_idx=1
    # Read until video is completed
    while(cap.isOpened()):
    # Capture frame-by-frame
        ret, frame = cap.read()
        if ret == True:
            print("Processing on frame "+str(frame_idx))
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            (img_ori,memo) = detect_per_frame(image_rgb)
            cv2.imwrite('out/'+str(frame_idx)+'output.jpg',img_ori)
            # Ghi dữ liệu JSON vào tệp
            with open('out/frame-'+ str(frame_idx)+'.json', 'w') as file:
                json.dump(memo, file,default=node_to_dict)
            frame_idx = frame_idx+1
        else:
            break
       
    # When everything done, release the video capture object
    cap.release()
    out.release()
    # Closes all the frames
    cv2.destroyAllWindows()
           

In [132]:
detect_from_video('sperm.avi')

Processing on frame 1
Processing on frame 2
Processing on frame 3
Processing on frame 4
[tensor(418.), tensor(699.), tensor(437.), tensor(699.)]
Processing on frame 5
Processing on frame 6
Processing on frame 7
[tensor(2.), tensor(699.), tensor(39.), tensor(699.)]
Processing on frame 8
[tensor(2.), tensor(699.), tensor(37.), tensor(699.)]
Processing on frame 9
[tensor(2.), tensor(699.), tensor(38.), tensor(699.)]
Processing on frame 10
Processing on frame 11
[tensor(1.), tensor(699.), tensor(37.), tensor(699.)]
Processing on frame 12
Processing on frame 13
Processing on frame 14
Processing on frame 15
[tensor(15.), tensor(699.), tensor(50.), tensor(699.)]
Processing on frame 16
Processing on frame 17
Processing on frame 18
Processing on frame 19
Processing on frame 20
[tensor(632.), tensor(699.), tensor(649.), tensor(699.)]
Processing on frame 21
Processing on frame 22
Processing on frame 23
Processing on frame 24
Processing on frame 25
Processing on frame 26
Processing on frame 27
[te